### Loading Libraries

In [1]:
import numpy as np
import cv2
from keras.models import load_model
import keras
import tensorflow
print(keras.__version__)
print(tensorflow.__version__)

'''
ERROR : load_model 'str' object has no attribute 'decode'

SOLVED:
pip uninstall h5py
pip install h5py==2.10.0
'''

model = load_model("yoonjungchoi_mnist_3.h5")

Using TensorFlow backend.


2.3.1
2.0.0


2022-10-23 11:22:14.860277: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 11:22:14.860462: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


### Options for webcam or video file

In [2]:
print("Option to support both live webcam [0], saved video file [1]")
user_input = int(input())
cp = 0
out = 0

if user_input == 0:
    cp = cv2.VideoCapture(0)
    w = round(cp.get(cv2.CAP_PROP_FRAME_WIDTH)) #1280
    h = round(cp.get(cv2.CAP_PROP_FRAME_HEIGHT)) #720
    fps = cp.get(cv2.CAP_PROP_FPS) #29
    #fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('webcam_mnist_output.mp4', fourcc, fps, (w, h))
    out_pre = cv2.VideoWriter('webcam_mnist_pre_output.mp4', fourcc, fps, (w, h))

elif user_input == 1:
    cp = cv2.VideoCapture('SID7062.mp4')
    w = round(cp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cp.get(cv2.CAP_PROP_FPS)
    frame_count = int(cp.get(cv2.CAP_PROP_FRAME_COUNT))
    print('duration sec- ' ,frame_count/fps)
    #fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('video_mnist_output.mp4', fourcc, fps, (w, h))
    out_pre = cv2.VideoWriter('video_mnist_pre_output.mp4', fourcc, fps, (w, h))


Option to support both live webcam [0], saved video file [1]


 1


duration sec-  12.566666666666666


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [3]:
# webcam size
cp.set(3, 5*128)
cp.set(4, 5*128)

#input mage size of model
SIZE = 28

before_cropped_digit=0

#cropped images for each digits
def extract_digit(frame, rect, pad = 10):
    x, y, w, h = rect
    
    #cropped image with padding
    cropped_digit = gray_img[y-pad:y+h+pad, x-pad:x+w+pad]
    #normalization
    cropped_digit = cropped_digit/255.0
    
    #crop with squared
    max_wh = max(w,h)
    w_pad = int((max_wh-w)/2)
    h_pad = int((max_wh-h)/2)
    cropped_digit_pad = gray_img[y-h_pad:y+h+h_pad, x-w_pad:x+w+w_pad].copy() 
    cropped_digit_pad = cropped_digit_pad/255.0

    #have certain shapes
    if cropped_digit_pad.shape[0] >= 32 and cropped_digit_pad.shape[1] >= 32:
    #if cropped_digit_pad is not None:
        #To train cropped image on model, SIZE should be 28*28
        #cropped_digit = cv2.resize(cropped_digit, (SIZE, SIZE))
        cropped_digit = cv2.resize(cropped_digit_pad, (SIZE, SIZE))
        #cv2.imshow('cropped_digit_pad',cropped_digit)
    else:
        #too small and return nothing
        return
    return cropped_digit;

def preprocessing(frame, tresh = 90):
    
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
    #adaptive here does better with variable lighting:
    gray_img = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, blockSize = 321, C = 28)
    return gray_img

    #-------canny is not stable-------#
    '''
    grayimage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
    cannyimage = cv2.Canny(grayimage, 100, 200)
    return cannyimage
    '''

### running

In [4]:
for i in range(1000):
    #print(i)
    ret, frame = cp.read(0)

    if ret==False:
        print('ret is false..')
        break;

    gray_img = preprocessing(frame)
    mnist_image = frame
    
    #------Requirement1: ROI-------#
    contours, _ = cv2.findContours(gray_img.copy(), cv2.RETR_EXTERNAL,
                                      cv2.CHAIN_APPROX_SIMPLE)

    rects = [cv2.boundingRect(contour) for contour in contours]
    rects = [rect for rect in rects if rect[2] >= 3 and rect[3] >= 8]

    
    #-----Requirement2: ready for preprocessing frame------#
    w2 = round(cp.get(cv2.CAP_PROP_FRAME_WIDTH))
    h2 = round(cp.get(cv2.CAP_PROP_FRAME_HEIGHT))
    prepro_image = np.ones((h2, w2), np.uint8) + 255
    prepro_image.fill(255)   
    prepro_image_to_bgr = cv2.cvtColor(prepro_image,cv2.COLOR_GRAY2BGR)
    
    #------Requirement3: draw rectangles with prediction from classifer------#
    for rect in rects:
        x, y, w, h = rect
        #print('x, y, w, h ', x, y, w, h )

        if i >= 0:
            mnist_frame = extract_digit(frame, rect, pad = 15)
                        
            if mnist_frame is not None: #and i % 25 == 0:
                mnist_frame = np.expand_dims(mnist_frame, 0)
                mnist_frame = np.expand_dims(mnist_frame, 3)
                
                result_arr = model.predict(mnist_frame)
                class_prediction = np.argmax(result_arr, axis=1)

                prediction = np.around(np.max(model.predict(mnist_frame, verbose = False)), 2)
                label = str(prediction) # if you want probabilities

                cv2.rectangle(mnist_image, (x, y), (x + w, y + h), color = (0, 255, 0), thickness=4)
                label = str(class_prediction)
                cv2.putText(mnist_image, label, (rect[0]+20, rect[1]-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            
            #-----Requirement2: add cropped images on preprocessing frame----#

                #create background square using longest length
                max_wh = max(w,h)
                square_bg = np.zeros((max_wh, max_wh), np.uint8)
                square_bg2 = np.zeros((max_wh, max_wh, 3), np.uint8)

                #calculate position
                w_start = int((max_wh-w)/2)
                h_start = int((max_wh-h)/2)
                w_end = int((max_wh+w)/2)
                h_end = int((max_wh+h)/2)

                #get digit from gray, origin
                cropped_gray_img = gray_img[y:y + h, x:x + w]
                cropped_img = frame[y:y + h, x:x + w]

                #add cropped image into background
                square_bg[h_start:h_end, w_start:w_end] = cropped_gray_img.copy()
                square_bg2[h_start:h_end, w_start:w_end] = cropped_img.copy()

                #cv2.imshow('square_bg2',square_bg2)
                #add squred preprocessing image into preprocessing_frame
                try:
                    prepro_image[y: y+max_wh, x: x+max_wh] = square_bg.copy()
                    mnist_image[y+200: y+200+max_wh, x: x+max_wh] = square_bg2.copy()
                    prepro_image_to_bgr = cv2.cvtColor(prepro_image,cv2.COLOR_GRAY2BGR)
                except:
                    pass

    out.write(mnist_image)
    out_pre.write(prepro_image_to_bgr)
    #cv2.imshow('Result', mnist_image)
    #cv2.imshow('preprocessing', prepro_image)
    cv2.imshow("YoonjungChoi", np.hstack([mnist_image, prepro_image_to_bgr]))

    if cv2.waitKey(1) & 0xFF==ord("q"):
        print('break')
        break

cp.release()
out.release()
out_pre.release()
print('release...!')
cv2.destroyAllWindows()
print('destroy windows...')
for i in range (1,5):
    cv2.waitKey(1)
print('destroy windows...!')

ret is false..
release...!
destroy windows...
destroy windows...!
